In [1]:
import numpy as np
import pandas as pd

In [2]:
path = r"C:\Users\enoch\OneDrive\Documents\GitHub\lang\dicts\chinese\integrated_chinese_1-2.tsv"
df = pd.read_csv(path, sep='\t', header=None)
df

,0,1,2
0,人,rén,person
1,刀,dāo,knife
2,力,lì,power
3,又,yòu,"right hand, again"
4,口,kǒu,mouth
...,...,...,...
863,瘦,shòu,"thin, skinny, lean"
864,爷爷,yéye,paternal grandfather
865,奶奶,nǎinai,paternal grandmother
866,烤鸭,kǎoyā,roast dusk


In [3]:
def unique_chars(col):
    all_chars = np.array([])
    for i, word in enumerate(col):
        for char in word:
            if char not in all_chars:
                all_chars = np.append(all_chars, char)
    return all_chars

In [4]:
total_chars = unique_chars(df[0])
print(total_chars)
with open("output.txt", "w") as file:
    for char in total_chars:
        file.write(char)
        file.write("\n")

['人' '刀' '力' '又' '口' '囗' '土' '夕' '大' '女' '子' '寸' '小' '工' '幺' '弓' '心' '戈'
 '手' '日' '月' '木' '水' '火' '田' '目' '示' '糸' '耳' '衣' '言' '贝' '走' '足' '金' '门'
 '隹' '雨' '食' '马' '一' '二' '三' '四' '五' '六' '七' '八' '九' '十' '百' '你' '好' '请'
 '问' '贵' '姓' '我' '呢' '姐' '叫' '什' '么' '名' '字' '先' '生' '李' '友' '王' '朋' '是'
 '老' '师' '吗' '不' '学' '也' '中' '国' '京' '北' '美' '纽' '约' '韩' '那' '的' '照' '片'
 '这' '爸' '妈' '个' '孩' '谁' '她' '男' '弟' '他' '哥' '儿' '有' '没' '高' '文' '家' '几'
 '两' '妹' '和' '做' '作' '律' '英' '都' '医' '白' '爱' '号' '星' '期' '天' '今' '年' '多'
 '岁' '吃' '饭' '怎' '样' '太' '…' '了' '谢' '喜' '欢' '菜' '还' '可' '们' '点' '半' '晚'
 '上' '见' '再' '现' '在' '刻' '事' '很' '忙' '明' '为' '因' '同' '认' '识' '周' '末' '打'
 '球' '看' '电' '视' '唱' '歌' '跳' '舞' '听' '音' '乐' '书' '对' '时' '候' '影' '常' '去'
 '外' '客' '昨' '所' '以' '久' '错' '想' '觉' '得' '意' '思' '只' '睡' '算' '找' '别' '呀'
 '进' '快' '来' '介' '绍' '下' '兴' '漂' '亮' '坐' '哪' '校' '喝' '茶' '咖' '啡' '吧' '要'
 '瓶' '起' '给' '杯' '玩' '图' '馆' '聊' '才' '回' '话' '喂' '就' '您' '位' '午' '间' '题'
 '开' '会' '节' '课' '级' '考' '试' '后' '空' '方' '便' '到' '办

C:\Users\enoch\AppData\Local\Temp\ipykernel_4884\2117195598.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if char not in all_chars:
